In [1]:
!pip install -q -U \
    langchain \
    langchain_community \
    transformers \
    sentence-transformers \
    faiss-cpu \
    torch \
    accelerate \
    bitsandbytes \
    huggingface_hub \
    ctransformers[cuda] \
    flask-ngrok \
    flask \
    jq \
    llama-cpp-python==0.2.76 \
    python-dotenv \
    ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.1/757.1 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import textwrap
import torch

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [3]:
from google.colab import drive
try:
    drive.mount('/content/drive')
except:
    print("Drive already mounted.")

Mounted at /content/drive


In [4]:
print("\n--- Loading and Processing Knowledge Base ---")

loader = JSONLoader(
    file_path="/content/drive/MyDrive/Model&data/diseases.json",
    jq_schema=".",
    text_content=False
)
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)



--- Loading and Processing Knowledge Base ---


In [5]:
print("Loading embedding model...")
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

print("Creating FAISS vector store...")
db = FAISS.from_documents(docs, embeddings)

# Retriever
retriever = db.as_retriever(search_kwargs={"k": 3})


Loading embedding model...


/tmp/ipython-input-3679667107.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating FAISS vector store...


In [6]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -q -U llama-cpp-python --no-cache-dir

# Install other necessary libraries
!pip install -q -U langchain langchain_community sentence-transformers faiss-cpu flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 195.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for llama-cpp-python
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (llama-cpp-python)


In [7]:
import torch
print(torch.cuda.is_available())

True


In [8]:
print("Loading LLM...")

model_name = "microsoft/phi-2"  # Example, you can change
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.2,
    top_p=0.95,
    repetition_penalty=1.2
)

llm = HuggingFacePipeline(pipeline=pipe)


Loading LLM...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-2518198027.py:21: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [9]:
prompt_template = """
### INSTRUCTIONS ###
You are a helpful and polite AI medical health assistant. Your job is to answer the user's question accurately and safely based ONLY on the information provided in the 'CONTEXT' section.

**Strict Rules:**
1.  **ONLY** use the information from the 'CONTEXT' section to answer the question.
2.  If the context is insufficient, you **MUST** state: "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
3.  **NEVER** invent information or use any knowledge outside of the provided context.
4.  **NEVER** include the sections "CONTEXT," "QUESTION," or "INSTRUCTIONS" in your final answer.
5.  **ALWAYS** provide the answer in a clear, easy-to-read markdown format (using bullet points for lists if appropriate).
6.  **ALWAYS** end your final response with the following disclaimer on a new line:
    "Disclaimer: I am an AI assistant and not a medical professional. Please consult a doctor for any health concerns."

### CONTEXT ###
{context}

### QUESTION ###
{question}

### HELPFUL ANSWER ###
"""


In [10]:
print("\n--- Setting up the RAG Chain ---")

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)



--- Setting up the RAG Chain ---


In [11]:
qa_chain.combine_documents_chain.llm_chain.prompt.template = prompt_template

In [12]:
NGROK_AUTHTOKEN = "32slYmXPQSoqvFAu8zakxn13ftS_3Rc3Xf9tiWCi2sMbKxPJA"

In [13]:
from pyngrok import ngrok
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
ngrok.set_auth_token(NGROK_AUTHTOKEN)
ngrok.kill()
public_url = ngrok.connect(5000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

@app.route("/ask", methods=['POST'])
def ask_chatbot():
    data = request.get_json()
    question = data.get("question", "")
    if not question:
        return jsonify({"error": "No question provided"}), 400

    print(f"Received question: {question}")
    result = qa_chain.invoke(question)
    print(f"Generated answer: {result['result']}")

    return jsonify({"answer": result['result']})

if __name__ == "__main__":
    app.run(port=5000)


 * ngrok tunnel "https://262676452d66.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Received question: how i can live a healthy life


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 20:50:54] "POST /ask HTTP/1.1" 200 -


Generated answer: 
### INSTRUCTIONS ###
You are a helpful and polite AI medical health assistant. Your job is to answer the user's question accurately and safely based ONLY on the information provided in the 'CONTEXT' section.

**Strict Rules:**
1.  **ONLY** use the information from the 'CONTEXT' section to answer the question.
2.  If the context is insufficient, you **MUST** state: "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
3.  **NEVER** invent information or use any knowledge outside of the provided context.
4.  **NEVER** include the sections "CONTEXT," "QUESTION," or "INSTRUCTIONS" in your final answer.
5.  **ALWAYS** provide the answer in a clear, easy-to-read markdown format (using bullet points for lists if appropriate).
6.  **ALWAYS** end your final response with the following disclaimer on a new line:
    "Disclaimer: I am an AI assistant and not a medical professional. Please consult a doctor for any health concerns

...

Generated answer: 
### INSTRUCTIONS ###
You are a helpful and polite AI medical health assistant. Your job is to answer the user's question accurately and safely based ONLY on the information provided in the 'CONTEXT' section.

**Strict Rules:**
1.  **ONLY** use the information from the 'CONTEXT' section to answer the question.
2.  If the context is insufficient, you **MUST** state: "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
3.  **NEVER** invent information or use any knowledge outside of the provided context.
4.  **NEVER** include the sections "CONTEXT," "QUESTION," or "INSTRUCTIONS" in your final answer.
5.  **ALWAYS** provide the answer in a clear, easy-to-read markdown format (using bullet points for lists if appropriate).
6.  **ALWAYS** end your final response with the following disclaimer on a new line:
    "Disclaimer: I am an AI assistant and not a medical professional. Please consult a doctor for any health concerns

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Received question: i am little tried didn't slept last night , what should i eat or drink to be energetic 


INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 20:53:40] "POST /ask HTTP/1.1" 200 -


Generated answer: 
### INSTRUCTIONS ###
You are a helpful and polite AI medical health assistant. Your job is to answer the user's question accurately and safely based ONLY on the information provided in the 'CONTEXT' section.

**Strict Rules:**
1.  **ONLY** use the information from the 'CONTEXT' section to answer the question.
2.  If the context is insufficient, you **MUST** state: "I'm sorry, I don't have enough information to answer that question based on my current knowledge base."
3.  **NEVER** invent information or use any knowledge outside of the provided context.
4.  **NEVER** include the sections "CONTEXT," "QUESTION," or "INSTRUCTIONS" in your final answer.
5.  **ALWAYS** provide the answer in a clear, easy-to-read markdown format (using bullet points for lists if appropriate).
6.  **ALWAYS** end your final response with the following disclaimer on a new line:
    "Disclaimer: I am an AI assistant and not a medical professional. Please consult a doctor for any health concerns